# Data Exploration

In [1]:
import pandas as pd

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from string import punctuation

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

## Test and Train

In [2]:
#Loading Data

train = pd.read_csv('products_sentiment_train.tsv', delimiter='\t')
test = pd.read_csv('products_sentiment_test.tsv', delimiter='\t')

print 'Train shape : ', train.shape
print 'Test shape : ', test.shape

Train shape :  (1999, 2)
Test shape :  (500, 2)


In [3]:
#Naming the columns

train.columns = ['text', 'target']
test.drop('Id', axis=1, inplace=True)

In [4]:
#Check

train.head()

text  target
0  i downloaded a trial version of computer assoc...       1
1  the wrt54g plus the hga7t is a perfect solutio...       1
2  i dont especially like how music files are uns...       0
3  i was using the cheapie pail ... and it worked...       1
4  you can manage your profile , change the contr...       1

In [5]:
#Check

test.head()

text
0  so , why the small digital elph , rather than ...
1  3/4 way through the first disk we played on it...
2  better for the zen micro is outlook compatibil...
3    6 . play gameboy color games on it with goboy .
4  likewise , i 've heard norton 2004 professiona...

##  Overview by the Target

In [6]:
print 'Number of texts with the target == 1 : ', train[train['target']==1].shape[0]
print ('The percentage : %0.1f' % (100.*train[train['target']==1].shape[0]/train.shape[0]))
print '\nRandom 5 samples:'
for t in train.loc[train.target==1, ['text']].sample(5).values:
    print t

Number of texts with the target == 1 :  1273
The percentage : 63.7

Random 5 samples:
['for the price it is a well spent investment ! ']
['you can also slow a song down & speed it up , which is cool too . ']
["two : it has a removeable / rechargeable battery , so you don 't have to send the thing in when the re !"]
['in a word , " awesome " is how i would describe this camera . ']
['the bags attach inside the champ easily and all one has to do is cram the diaper down and flip the handle .']


In [7]:
print 'Number of texts with the target == 0 : ', train[train['target']==0].shape[0]
print ('The percentage : %0.1f' % (100.*train[train['target']==0].shape[0]/train.shape[0]))
print '\nRandom 5 samples:'
for t in train.loc[train.target==0, ['text']].sample(5).values:
    print t

Number of texts with the target == 0 :  726
The percentage : 36.3

Random 5 samples:
['i would imagine this is a problem with any camera of a compact nature .']
["one beef is the scroll , it does n't work as well as i 'd like . "]
["software - music match jukebox is n't the greatest , the search funtion is n't fast even when accessing it with the hotkey shortcut . "]
['its quiet . ']
["however , it needs to be quite quiet for it to work at all -- dosn 't work at all in the car , for example ."]


# Preprocessing

## Lowering 

In [8]:
train['lowered_text'] = train['text'].apply(lambda x: x.lower())
test['lowered_text'] = test['text'].apply(lambda x: x.lower())

## Removing Digits and Special Characters

In [9]:
def remove_digits(text):
    return re.sub("[\d+-/_\"]", "", text.lower())

In [10]:
train['treated_text'] = train['lowered_text'].apply(lambda x: remove_digits(x))
test['treated_text'] = test['lowered_text'].apply(lambda x: remove_digits(x))

## Replacing Contractions

In [11]:
contraction_mapping = {"n't": "not", "'t": "not", "'d": "would", "'ll": "will", "'s": "is", 
                       "'ve": "have", "'m": "am", "'re": "are"}

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [12]:
train['treated_text'] = train['treated_text'].apply(lambda x: clean_contractions(x, contraction_mapping))
test['treated_text'] = test['treated_text'].apply(lambda x: clean_contractions(x, contraction_mapping))

## Replacing Mispells

In [13]:
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling'}

def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

In [14]:
train['treated_text'] = train['treated_text'].apply(lambda x: correct_spelling(x, mispell_dict))
test['treated_text'] = test['treated_text'].apply(lambda x: correct_spelling(x, mispell_dict))

## Removing Punctuation and Lemmatizing

In [15]:
punctuation = [char for char in punctuation if char not in ['?', '!']]

def final_prep(text):
    words = text.split()               
    meaningful_words = [WordNetLemmatizer().lemmatize(w) for w in words if w not in punctuation]   
    return(" ".join( meaningful_words ))   

In [16]:
train['treated_text'] = train['treated_text'].apply(lambda x: final_prep(x))
test['treated_text'] = test['treated_text'].apply(lambda x: final_prep(x))

## Checking

In [17]:
for q in train['treated_text'].sample(5):
    print q, '\n'

with bluetooth off the phone last almost twice a long 

but this camera is great ! 

also if you do manage to get it installed any problem with liveupdate or with system work will also affect the functionality and or the process of updating internet security 

the smooth operation variable speed and a set of good jasada bit will provide just the right combination for great predictable result 

the auto setting doe a great job most of the time but there are manual setting for those who want them 



In [18]:
for q in test['treated_text'].sample(5):
    print q, '\n'

the volume range is great so you can blast it if you want 

before asking for the zen xtra specifically for christmas i researched many different mp player but finally decided on this one and now that i have it i could not be happier and i am convinced it is the best one out there 

well linksys wa one of the original big player in the router industry 

this ha happened with several different dvd is and occasionally if we keep trying something it will work when it did not work one time before 

verdict had i not needed something with a tad bit more space i consider this player fair game 



# Classifier Selection

## CountVec + Logistic Regression

In [19]:
count_lr_pipe = Pipeline([
    ("vectorizer", CountVectorizer(analyzer='word', ngram_range=(1,3))),
    ("classifier", LogisticRegression(class_weight='balanced'))])

In [20]:
count_lr_cv = cross_val_score(count_lr_pipe, train['treated_text'], train['target'], scoring='accuracy', cv=5)
print count_lr_cv.mean(), count_lr_cv.std()

/Users/Bai/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7878966931043319 0.006490023507407363


## Tf-Idf + Logistic Regression

In [22]:
tfidf_lr_pipe = Pipeline([
    ("vectorizer", TfidfVectorizer(analyzer='word', ngram_range=(1,3))),
    ("classifier", LogisticRegression(class_weight='balanced'))])

In [23]:
tfidf_lr_cv = cross_val_score(tfidf_lr_pipe, train['treated_text'], train['target'], scoring='accuracy', cv=5)
print tfidf_lr_cv.mean(), tfidf_lr_cv.std()

0.7863891868074175 0.005966513656341954


## Tf-Idf + Logistic Regression + StopWords

In [24]:
stops = set(stopwords.words("english"))  

tfidf_lr_s_pipe = Pipeline([
    ("vectorizer", TfidfVectorizer(analyzer='word', ngram_range=(1,3), stop_words=stops)),
    ("classifier", LogisticRegression(class_weight='balanced'))])

In [25]:
tfidf_lr_s_cv = cross_val_score(tfidf_lr_s_pipe, train['treated_text'], train['target'], scoring='accuracy', cv=5)
print tfidf_lr_s_cv.mean(), tfidf_lr_s_cv.std()

0.7513515365721036 0.016547232790117102


## Tf-Idf + LinearSVC

In [26]:
tfidf_svc_pipe = Pipeline([
    ("vectorizer", TfidfVectorizer(analyzer='word', ngram_range=(1,3))),
    ("classifier", LinearSVC(C=2.3))])

In [27]:
tfidf_svc_cv = cross_val_score(tfidf_svc_pipe, train['treated_text'], train['target'], scoring='accuracy', cv=5)
print tfidf_svc_cv.mean(), tfidf_svc_cv.std()

0.7928917305733161 0.007446575777154651


##  Tf-Idf + Random Forest

In [28]:
tfidf_rf_pipe = Pipeline([
    ("vectorizer", TfidfVectorizer(analyzer='word', ngram_range=(1,3))),
    ("classifier", RandomForestClassifier(n_estimators = 50, class_weight='balanced'))])

In [29]:
tfidf_rf_cv = cross_val_score(tfidf_rf_pipe, train['treated_text'], train['target'], scoring='accuracy', cv=5)
print tfidf_rf_cv.mean(), tfidf_rf_cv.std()

0.7453690304314402 0.005932628451406857


# Predictions

In [30]:
ans = pd.read_csv('products_sentiment_sample_submission.csv')

In [31]:
model = tfidf_svc_pipe.fit(train['treated_text'], train['target'])
ans['y'] = model.predict(test['treated_text'])

In [32]:
ans.to_csv('answer.csv', index=False)